### 1. system prompt, human prompt 작성해보기

In [1]:
from langchain.prompts import ChatMessagePromptTemplate, ChatPromptTemplate, SystemMessagePromptTemplate, HumanMessagePromptTemplate

In [2]:
# system 메시지 구성
system_message = "당신은 {language} 선생님입니다. {language} 로 간단하게 답변해주세요"
system_prompt = SystemMessagePromptTemplate.from_template(system_message)
system_prompt

SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['language'], template='당신은 {language} 선생님입니다. {language} 로 간단하게 답변해주세요'))

In [3]:
# human 메시지 구성
human_message = "{text} 에 대해서 얘기해줘"
human_prompt = HumanMessagePromptTemplate.from_template(human_message)
human_prompt

HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['text'], template='{text} 에 대해서 얘기해줘'))

In [4]:
# 최종 프롬프트 구성
chat_prompt = ChatPromptTemplate.from_messages(
    [system_prompt, 
     human_prompt]
    )

In [5]:
# 모델 설정
from langchain.chat_models import ChatOllama
model = ChatOllama(model="llama3.1:latest")

In [6]:
# 아웃풋 파서 설정
from langchain_core.output_parsers import StrOutputParser
outputparser = StrOutputParser()

In [7]:
# 체인 설정
chain = chat_prompt | model | outputparser

In [8]:
result = chain.stream({"language" : "english", "text" : "대한민국의 수도는어디인가요?"})
for chunk in result:
    print(chunk, end="", flush=True)

서울입니다.

### 2. 퓨샷 프롬프트 사용해보기

In [9]:
from langchain_core.prompts.few_shot import FewShotPromptTemplate
from langchain_core.prompts import PromptTemplate

In [10]:
examples = [
    {"question": "안녕하세요를 경상도 사투리로 어떻게 말하나요?",
     "answer": "안녕하이소"},
    {"question": "맛있게 먹어를 경상도 사투리로 어떻게 말하나요?",
     "answer" : "마이 무라"},
    {"question": "지금 뭐하고 있어?를 경상도 사투리로 어떻게 말하나요?",
     "answer" : "지금 뭐하노?"},    
]

In [11]:
example_prompt = PromptTemplate.from_template(
    """
질문내용 :
{question}를 경상도 사투리로 어떻게 말하나요?

사투리 :
{answer}"""
)
example_prompt

PromptTemplate(input_variables=['answer', 'question'], template='\n질문내용 :\n{question}를 경상도 사투리로 어떻게 말하나요?\n\n사투리 :\n{answer}')

In [12]:
prompt = FewShotPromptTemplate(
    examples=examples,
    example_prompt=example_prompt,
    suffix="""질문내용 :
    {question}
    사투리 : """,
    input_variables=['question']
)

In [13]:
question = "안녕하세요 지금 뭐하고 있어요?"
question_prompt = prompt.format(question=question)
print(question_prompt)


질문내용 :
안녕하세요를 경상도 사투리로 어떻게 말하나요?를 경상도 사투리로 어떻게 말하나요?

사투리 :
안녕하이소


질문내용 :
맛있게 먹어를 경상도 사투리로 어떻게 말하나요?를 경상도 사투리로 어떻게 말하나요?

사투리 :
마이 무라


질문내용 :
지금 뭐하고 있어?를 경상도 사투리로 어떻게 말하나요?를 경상도 사투리로 어떻게 말하나요?

사투리 :
지금 뭐하노?

질문내용 :
    안녕하세요 지금 뭐하고 있어요?
    사투리 : 


In [14]:
# chain 설정
chain = prompt | model | StrOutputParser()

result = chain.stream(
    {"question", "안녕하세요 지금 뭐하고 있어요?"}
)

for chunk in result:
    print(chunk, end="", flush=True)

경상도 사투리와 원어민 한국인의 대화는 다음과 같습니다.

Q: 안녕하세요, 지금 뭐하고 있어요?
A: 하이소, 무라하노?

 Q: 안녕하세요, 맛있게 먹었어요.
 A: 마무르, 미안하이요!

 Q: 안녕하세요, 지인이 왔어요.
 A: 하이소, 잘했어?

### 3. 다양한 출력파서를 사용해보기
- pydandtic
- jsonoutputparser
- comma
- pandasDataframe
- Datetime

In [15]:
from langchain.output_parsers import PydanticOutputParser
from langchain_core.pydantic_v1 import BaseModel, Field


# 1. pydantic 방식으로 응답 결과 받도록 해보기
# pydantic 클래스 정의
class UserRequest(BaseModel):
    date : str = Field(description="요청 시간")
    ask_type : str = Field(description="문의 종류")
    request_type : str = Field(description="요청 종류")
    expected_type : str = Field(description="기대되는 대응")
    text: str

# pydantic parser 설정
parser = PydanticOutputParser(pydantic_object=UserRequest)

In [16]:
parser.get_format_instructions()

'The output should be formatted as a JSON instance that conforms to the JSON schema below.\n\nAs an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}\nthe object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.\n\nHere is the output schema:\n```\n{"properties": {"date": {"title": "Date", "description": "\\uc694\\uccad \\uc2dc\\uac04", "type": "string"}, "ask_type": {"title": "Ask Type", "description": "\\ubb38\\uc758 \\uc885\\ub958", "type": "string"}, "request_type": {"title": "Request Type", "description": "\\uc694\\uccad \\uc885\\ub958", "type": "string"}, "expected_type": {"title": "Expected Type", "description": "\\uae30\\ub300\\ub418\\ub294 \\ub300\\uc751", "type": "string"}, "text": {"title": "Text", "type": "string"}}, "required": ["date", "ask_type", "request_type", "e

In [17]:
# prompt 구성
prompt = PromptTemplate.from_template(
    """당신은 물건을 판매하는 판매자야. 아래의 질문에 적절한 대응을 하도록 해.
    question:
    {text}

    FORMAT:
    {format}"""
)

In [18]:
prompt = prompt.partial(format=parser.get_format_instructions())
prompt

PromptTemplate(input_variables=['text'], partial_variables={'format': 'The output should be formatted as a JSON instance that conforms to the JSON schema below.\n\nAs an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}\nthe object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.\n\nHere is the output schema:\n```\n{"properties": {"date": {"title": "Date", "description": "\\uc694\\uccad \\uc2dc\\uac04", "type": "string"}, "ask_type": {"title": "Ask Type", "description": "\\ubb38\\uc758 \\uc885\\ub958", "type": "string"}, "request_type": {"title": "Request Type", "description": "\\uc694\\uccad \\uc885\\ub958", "type": "string"}, "expected_type": {"title": "Expected Type", "description": "\\uae30\\ub300\\ub418\\ub294 \\ub300\\uc751", "type": "string"}, "text": {"title": "Text", "

In [19]:
# chain 구성
chain = prompt | model | parser

In [20]:
result = chain.invoke({'text': "나는 지금 7월 29일에 산 티셔츠가 하자가 있어서 이것을 반품을 받으려고 해"})

In [21]:
# 결과 확인해보기
for item in dict(result).items():
    print(item)

('date', '7월 29일')
('ask_type', '반품')
('request_type', '티셔츠')
('expected_type', '하자')
('text', '아래의 티셔츠는 7월 29일에 구입한 제품입니다. 하자가 있는지 확인해 주세요.')


### 4. 메모리 사용법 익혀보기

In [22]:
# dic = {"user1": {"1": {},
#                  "2": {},
#                  },
#         "user2" :{}
                 
#                  }

In [23]:
from langchain.memory import ConversationBufferMemory

memory = ConversationBufferMemory()

memory.save_context(
    inputs={
        "human" : "안녕하세요 여러분"
    },
    outputs={
        "ai": "안녕하세요. 어떻게 도와드릴까요? 오늘 하루는 어떠셨나요?"
    }
)

In [24]:
memory

ConversationBufferMemory(chat_memory=InMemoryChatMessageHistory(messages=[HumanMessage(content='안녕하세요 여러분'), AIMessage(content='안녕하세요. 어떻게 도와드릴까요? 오늘 하루는 어떠셨나요?')]))

In [25]:
from langchain import ConversationChain

memory = ConversationBufferMemory()
conversation = ConversationChain(
    llm=model,
    verbose=True,
    memory=memory
)

c:\Users\Admin\anaconda3\envs\gumi_env310\lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The class `ConversationChain` was deprecated in LangChain 0.2.7 and will be removed in 1.0. Use RunnableWithMessageHistory: https://api.python.langchain.com/en/latest/runnables/langchain_core.runnables.history.RunnableWithMessageHistory.html instead.
  warn_deprecated(


In [26]:
# 첫대화 시작
conversation.predict(input="안녕하세요 여러분. 한글로 대화 해주세요")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: 안녕하세요 여러분. 한글로 대화 해주세요
AI:

> Finished chain.


"네! 반갑습니다. 한국어로 이야기를 하고 싶은군요? 제가 도와드릴 수 있을 것 같아요! 한국에 사는 사람들도 저한테 이렇게 물어보면 다들 안녕하세요 여러분. 한글로 대화 해주세요고 하더라구요. 내부적으로 한글 처리를 위해 NFD(normalization form decomposition)과 NFC(normalization form composition)을 사용해왔어요. 그리고 '안녕하세요'는 한국의 인사말로, '여러분'은 저희가 서로에게 호칭하는 말이에요. 이 외에도 '잘 지내셨어요?' 같은 대화 예를 가지고 있어요!"

In [27]:
# 두번째 대화
conversation.predict(input="내가 아까 뭐라고 했죠?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: 안녕하세요 여러분. 한글로 대화 해주세요
AI: 네! 반갑습니다. 한국어로 이야기를 하고 싶은군요? 제가 도와드릴 수 있을 것 같아요! 한국에 사는 사람들도 저한테 이렇게 물어보면 다들 안녕하세요 여러분. 한글로 대화 해주세요고 하더라구요. 내부적으로 한글 처리를 위해 NFD(normalization form decomposition)과 NFC(normalization form composition)을 사용해왔어요. 그리고 '안녕하세요'는 한국의 인사말로, '여러분'은 저희가 서로에게 호칭하는 말이에요. 이 외에도 '잘 지내셨어요?' 같은 대화 예를 가지고 있어요!
Human: 내가 아까 뭐라고 했죠?
AI:

> Finished chain.


'네! 제가 잘못 기억하고 있었는데, 실제로 처음에 "안녕하세요 여러분"이라고 하시지 않았어요. 저는 한국어 음성 인식 모델을 사용하기 때문에, 내부적으로 이전의 대화를 기록하고 있어요. 그 전에 대한 정보를 제공해드릴 수 있어요.\n\n처음에 제가 이해한 결과는 다음과 같았습니다:\n\n* "안녕하세요 여러분"이라고 처음으로 한글로 이야기하였습니다.\n* 한국어로 이야기를 하고 싶은 것 같습니다!\n* 저도 도움이 될 것 같아서 같이 이야기하고 싶습니다!\n\n그리고, 내부적으로는 NFD와 NFC를 사용하여 한글 처리를 하였고, "안녕하세요"는 인사말로, "여러분"은 서로에게 호칭하는 말이라고 알려드렸어요.'

In [28]:
print(memory.buffer)

Human: 안녕하세요 여러분. 한글로 대화 해주세요
AI: 네! 반갑습니다. 한국어로 이야기를 하고 싶은군요? 제가 도와드릴 수 있을 것 같아요! 한국에 사는 사람들도 저한테 이렇게 물어보면 다들 안녕하세요 여러분. 한글로 대화 해주세요고 하더라구요. 내부적으로 한글 처리를 위해 NFD(normalization form decomposition)과 NFC(normalization form composition)을 사용해왔어요. 그리고 '안녕하세요'는 한국의 인사말로, '여러분'은 저희가 서로에게 호칭하는 말이에요. 이 외에도 '잘 지내셨어요?' 같은 대화 예를 가지고 있어요!
Human: 내가 아까 뭐라고 했죠?
AI: 네! 제가 잘못 기억하고 있었는데, 실제로 처음에 "안녕하세요 여러분"이라고 하시지 않았어요. 저는 한국어 음성 인식 모델을 사용하기 때문에, 내부적으로 이전의 대화를 기록하고 있어요. 그 전에 대한 정보를 제공해드릴 수 있어요.

처음에 제가 이해한 결과는 다음과 같았습니다:

* "안녕하세요 여러분"이라고 처음으로 한글로 이야기하였습니다.
* 한국어로 이야기를 하고 싶은 것 같습니다!
* 저도 도움이 될 것 같아서 같이 이야기하고 싶습니다!

그리고, 내부적으로는 NFD와 NFC를 사용하여 한글 처리를 하였고, "안녕하세요"는 인사말로, "여러분"은 서로에게 호칭하는 말이라고 알려드렸어요.


In [29]:
for chat in memory.buffer_as_messages:
    print(chat.content)

안녕하세요 여러분. 한글로 대화 해주세요
네! 반갑습니다. 한국어로 이야기를 하고 싶은군요? 제가 도와드릴 수 있을 것 같아요! 한국에 사는 사람들도 저한테 이렇게 물어보면 다들 안녕하세요 여러분. 한글로 대화 해주세요고 하더라구요. 내부적으로 한글 처리를 위해 NFD(normalization form decomposition)과 NFC(normalization form composition)을 사용해왔어요. 그리고 '안녕하세요'는 한국의 인사말로, '여러분'은 저희가 서로에게 호칭하는 말이에요. 이 외에도 '잘 지내셨어요?' 같은 대화 예를 가지고 있어요!
내가 아까 뭐라고 했죠?
네! 제가 잘못 기억하고 있었는데, 실제로 처음에 "안녕하세요 여러분"이라고 하시지 않았어요. 저는 한국어 음성 인식 모델을 사용하기 때문에, 내부적으로 이전의 대화를 기록하고 있어요. 그 전에 대한 정보를 제공해드릴 수 있어요.

처음에 제가 이해한 결과는 다음과 같았습니다:

* "안녕하세요 여러분"이라고 처음으로 한글로 이야기하였습니다.
* 한국어로 이야기를 하고 싶은 것 같습니다!
* 저도 도움이 될 것 같아서 같이 이야기하고 싶습니다!

그리고, 내부적으로는 NFD와 NFC를 사용하여 한글 처리를 하였고, "안녕하세요"는 인사말로, "여러분"은 서로에게 호칭하는 말이라고 알려드렸어요.


### 4-2. conversationChain 과 conversationbufferwindowmemory 같이 사용해보기

In [30]:
from langchain.memory import ConversationBufferWindowMemory

# 가장 최근의 대화내용 2개만 기억
memory = ConversationBufferWindowMemory(k=2)
conversation = ConversationChain(
    llm=model,
    verbose=True,
    memory=memory
)

In [31]:
# 첫대화 시작
conversation.predict(input="안녕하세요 여러분. 한글로 대화 해주세요")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: 안녕하세요 여러분. 한글로 대화 해주세요
AI:

> Finished chain.


"Hello! Nice to chat with you in Korean. My language processing algorithms are specifically designed for Hangul characters, and I'm happy to converse with you in a natural-sounding way. I can understand and respond to the nuances of the Korean language, from simple phrases to complex sentences.\n\nBy the way, did you know that Hangul is the official writing system of Korea, consisting of 14 consonants and 10 vowels? It's a unique script that's both phonetic and logographic in nature. I can even generate text for you in Hangul if you'd like! Would you like to practice some Korean vocabulary or discuss something specific?"

In [32]:
# 두번째 대화
conversation.predict(input="내가 아까 뭐라고 했죠?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: 안녕하세요 여러분. 한글로 대화 해주세요
AI: Hello! Nice to chat with you in Korean. My language processing algorithms are specifically designed for Hangul characters, and I'm happy to converse with you in a natural-sounding way. I can understand and respond to the nuances of the Korean language, from simple phrases to complex sentences.

By the way, did you know that Hangul is the official writing system of Korea, consisting of 14 consonants and 10 vowels? It's a unique script that's both phonetic and logographic in nature. I can even generate text for you in Hangul if you'd like! Would you like to practice some Korean vocabulary or discuss something specific?
Human: 내가 아까 뭐라고 했

'You just said "" (Annyeonghaseyo) which is the traditional way of greeting someone in Korean, literally meaning "be at peace" or "peace be with you". It\'s a polite way to say hello, especially when meeting someone for the first time. I\'m happy to chat with you in Hangul and respond accordingly! If you\'d like, we can discuss something specific or practice some vocabulary together?'

In [33]:
# 세번째 대화
conversation.predict(input="우리 세번째 대화 내용입니다")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: 안녕하세요 여러분. 한글로 대화 해주세요
AI: Hello! Nice to chat with you in Korean. My language processing algorithms are specifically designed for Hangul characters, and I'm happy to converse with you in a natural-sounding way. I can understand and respond to the nuances of the Korean language, from simple phrases to complex sentences.

By the way, did you know that Hangul is the official writing system of Korea, consisting of 14 consonants and 10 vowels? It's a unique script that's both phonetic and logographic in nature. I can even generate text for you in Hangul if you'd like! Would you like to practice some Korean vocabulary or discuss something specific?
Human: 내가 아까 뭐라고 했

"This is indeed our third conversation! Congratulations on keeping track of our dialogue. As for the content of this third conversation, I'm happy to continue exploring topics with you.\n\nTo confirm, we've already discussed Hangul and your initial greeting in Korean. If you'd like to explore something specific or practice more vocabulary, please feel free to let me know. Alternatively, we can simply keep chatting about general topics. What would you like to talk about next?"

In [34]:
print(memory.buffer)

Human: 내가 아까 뭐라고 했죠?
AI: You just said "" (Annyeonghaseyo) which is the traditional way of greeting someone in Korean, literally meaning "be at peace" or "peace be with you". It's a polite way to say hello, especially when meeting someone for the first time. I'm happy to chat with you in Hangul and respond accordingly! If you'd like, we can discuss something specific or practice some vocabulary together?
Human: 우리 세번째 대화 내용입니다
AI: This is indeed our third conversation! Congratulations on keeping track of our dialogue. As for the content of this third conversation, I'm happy to continue exploring topics with you.

To confirm, we've already discussed Hangul and your initial greeting in Korean. If you'd like to explore something specific or practice more vocabulary, please feel free to let me know. Alternatively, we can simply keep chatting about general topics. What would you like to talk about next?


### 5. 대화 챗봇 만들어보기 - system 설정도 함께, 대화내용도 기록

In [35]:
from langchain_core.prompts import MessagesPlaceholder

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system", "당신은 친절한 상담사야. 상담 내용에 맞게 한국어로 {number}자 내외로 답변을 해줘"
        ),
        MessagesPlaceholder(variable_name="history"), # history라는 이름의 메세지 저장 객체
        (
            "human", "{input}"
        )
    ]
)

chain = prompt | model

In [36]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

In [45]:
session_store = {} # 메시지 내용을 저장할 딕셔너리

def get_session_history(session_ids : str) -> BaseChatMessageHistory:
    print(session_ids)

    if session_ids not in session_store:
        session_store[session_ids] = ChatMessageHistory()
    return session_store[session_ids]

In [46]:
with_message_history = (
    RunnableWithMessageHistory(
        runnable=chain,
        get_session_history=get_session_history,
        input_messages_key="input",
        history_messages_key='history',
    )
)

In [47]:
with_message_history

RunnableWithMessageHistory(bound=RunnableBinding(bound=RunnableBinding(bound=RunnableAssign(mapper={
  history: RunnableBinding(bound=RunnableLambda(_enter_history), config={'run_name': 'load_history'})
}), config={'run_name': 'insert_history'})
| RunnableBranch(branches=[(RunnableBinding(bound=RunnableLambda(_is_not_async), config={'run_name': 'RunnableWithMessageHistoryInAsyncMode'}), RunnableBinding(bound=ChatPromptTemplate(input_variables=['history', 'input', 'number'], input_types={'history': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]]}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['number'], template='당신은 친절한 상담사야. 상담 내용에 맞게 한국어로 {number}자 내외로 답변을 해줘')), MessagesPlaceholder(variable_name='history'), HumanMess

In [48]:
result = with_message_history.invoke(
    {"number": "100", "input": "요새 코딩 강의 듣느라 너무 즐거워요"},
    config={"configurable": {"session_id": "abc123"}},
)

abc123
